In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
from folium.plugins import MarkerCluster

import altair as alt
alt.renderers.enable('notebook')
from vega_datasets import data

from math import pi

pd.set_option('display.max_rows', 500)

# code reference at blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9
# transit districts source: www1.nyc.gov/assets/nypd/downloads/pdf/maps/transit-district-map.pdf

In [53]:
# type of graph in 'json' format
json1_file = open('stackedBar.json')
json1_str = json1_file.read()

In [58]:
# load csv of all station information
subway_data = pd.read_csv("Stations.csv")

# need to get lat/long of all stations to draw circle
# need to figure out how to make interactable (default pics)

In [59]:
locations = subway_data.groupby("GTFS Latitude").first()
locations = locations.loc[:, ["GTFS Longitude",
                              "Stop Name",
                              "Borough"]]
locations.groupby('Stop Name')
locations.head()
locations.drop_duplicates(subset ='Stop Name',keep='first',inplace=True)
locations = locations.sort_values(by='Stop Name')

In [60]:
highCountStations = ['59 St - Columbus Circle','14 St','125 St','86 St','167 St','Jackson Av','Court Sq','Rockaway Blvd','Franklin Av','Atlantic Av','Utica Av','Newkirk Av']
highCountStations.sort()
lowCountStations = ['72 St','Franklin St','157 St','51 St','Mt Eden Av','Freeman St','Astoria Blvd','Beach 98 St','Carroll St','Winthrop St','Norwood Av','Ditmas Av']

In [62]:
highLocations = locations[locations['Stop Name'].isin(highCountStations)]
highSpots = ['High','High','High','High','High','High','High','High','High','High','High','High']
highLocations.sort_values(by='Stop Name')
highLocations['Cop Sightings'] = highSpots

/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [64]:
lowLocations = locations[locations['Stop Name'].isin(lowCountStations)]
lowLocations.sort_values(by='Stop Name')
lowSpots = ['Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low']
lowLocations['Cop Sightings'] = lowSpots

/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [42]:
transitDistricts = [3,2,3,11,4,1,1,1,20,32,23,30,30,34,32,2,12,12,11,32,33,23,33,32]

In [65]:
lowHighLocations = [lowLocations,highLocations]
lowHigh = pd.concat(lowHighLocations)
lowHigh = lowHigh.sort_values(by='Stop Name',ascending=True)
lowHigh['Transit District'] = transitDistricts

In [66]:
# plot circles at each station
def plot_stations(station):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=10,
                            tiles="openstreetmap",
                            width='100%')
    
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        #print(row.isin(lowLocations['Stop Name']))
        # number of spotted officers plotted
        
        # generate popup message shown on click
        popup_text = "stop name: {}"
        popup_text = popup_text.format(row["Stop Name"])
        #popup_text = json1_str
        #popup_text = folium.Popup(max_width=550).add_child(
                                  #folium.VegaLite(json1_str, width=550, height=250))
        
        # radius of circles
        radius = 10
        # color of circles (can change conditionally, by borough)
        if(row["Borough"] == 'Bx'):
            #Bronx = red
            color='#FF0000'
        elif(row["Borough"] == 'Bk'):
            #Brooklyn=yellow
            color='#ffff00'
        elif(row["Borough"] == 'M'):
            #Manhattan=green
            color='#00ff00'
        elif(row["Borough"] == 'Q'):
            #Queens=orange
            color='#ffa500'
        elif(row["Borough"] == 'SI'):
            #Staten Island=Purple
            color="#0A8A9F"
        else:
            #Missing Borough colored blue
            color='#800080'
        # color for low Locations is: purple
        for i in (lowLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#0A8A9F'
        for i in (highLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#8b0000'
        #print(color)
        #color='#00ff00'
        folium.CircleMarker(location=(index,
                                      row["GTFS Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=False,
                            fill_color=color,
                            tooltip=popup_text,
                            clustered_marker=True).add_to(folium_map)

    return folium_map

In [67]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_name(station):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=10,
                            tiles="openstreetmap",
                            width='100%')
    folium.TileLayer('stamentoner').add_to(folium_map)
    folium.TileLayer('stamenterrain').add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        #print(row.isin(lowLocations['Stop Name']))
        # number of spotted officers plotted
        
        # generate popup message shown on click
        #popup_text = "stop name: {}"
        #popup_text = popup_text.format(row["Stop Name"])
        popup_text = json1_str
        popup_text = folium.Popup(max_width=550).add_child(
                                  folium.VegaLite(json1_str, width=550, height=250))
        
        # tooltip is text displayed when only hovering over popup
        tooltip_text = "{}"
        tooltip_text = tooltip_text.format(row["Stop Name"])
        # radius of circles
        radius = 10
        # color of circles (can change conditionally, by borough)
        #if(row["Borough"] == 'Bx'):
            #Bronx = red
            #color='#FF0000'
        #elif(row["Borough"] == 'Bk'):
            #Brooklyn=yellow
            #color='#ffff00'
        #elif(row["Borough"] == 'M'):
            #Manhattan=green
            #color='#00ff00'
        #elif(row["Borough"] == 'Q'):
            #Queens=orange
            #color='#ffa500'
        #elif(row["Borough"] == 'SI'):
            #Staten Island=Purple
            #color="#0A8A9F"
        #else:
            #Missing Borough colored blue
            #color='#800080'
        # color for low Locations is: purple
        for i in (lowLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#008b46'
        for i in (highLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#8b0000'
        #print(color)
        #color='#00ff00'
        folium.CircleMarker(location=(index,
                                      row["GTFS Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=False,
                            fill_color=color,
                            tooltip=tooltip_text,
                            clustered_marker=True).add_to(folium_map)

    return folium_map

In [68]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_transit_district(station):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=10,
                            tiles="stamentoner",
                            width='100%')
    folium.TileLayer('openStreetMap').add_to(folium_map)
    folium.TileLayer('stamenterrain').add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        #print(row.isin(lowLocations['Stop Name']))
        # number of spotted officers plotted
        
        # generate popup message shown on click
        #popup_text = "stop name: {}"
        #popup_text = popup_text.format(row["Stop Name"])
        popup_text = json1_str
        popup_text = folium.Popup(max_width=550).add_child(
                                  folium.VegaLite(json1_str, width=550, height=250))
        
        # tooltip is text displayed when only hovering over popup
        tooltip_text = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_text = tooltip_text.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        radius = 10
        # color of circles (can change conditionally)
        
        # color for low Locations is: purple
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#964b00'
        if(row['Transit District']==11):
            #pink
            color = '#964b00'
        if(row['Transit District']==4):
            #black
            color = '#964b00'
        if(row['Transit District']==1):
            #red
            color = '#964b00'
        if(row['Transit District']==20):
            #blue
            color = '#964b00'
        if(row['Transit District']==32):
            #violet
            color = '#964b00'
        if(row['Transit District']==23):
            #orange
            color = '#964b00'
        if(row['Transit District']==30):
            #teal
            color = '#964b00'
        if(row['Transit District']==34):
            #brown
            color = '#964b00'
        if(row['Transit District']==12):
            #yellow
            color = '#964b00'
        if(row['Transit District']==33):
            #purple
            color = '#964b00'
        
        #print(color)
        #color='#00ff00'
        folium.CircleMarker(location=(index,
                                      row["GTFS Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_text,
                            clustered_marker=True).add_to(folium_map)

    return folium_map

In [69]:
# call function plot_stations of each subway stop
#plot_stations(locations)

In [70]:
# CREATES MAIN MAP
plot_stations_colored_by_transit_district(lowHigh)

In [97]:
folium_map = plot_stations_colored_by_transit_district(lowHigh)

In [98]:
#folium_map.save("ClickableSubwayMapStackedBarColoredTransitDistrict.html")

In [17]:
#folium_map.save("testGeoJson.html")